In [ ]:
!nvidia-smi

Fri Apr 22 01:18:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install simpletransformers
!pip install wandb
!pip install PyTDC

     |████████████████████████████████| 4.0 MB 15.9 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
     |████████████████████████████████| 895 kB 49.5 MB/s 
     |████████████████████████████████| 596 kB 65.7 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 249 kB 14.2 MB/s 
     |████████████████████████████████| 1.8 MB 61.7 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 325 kB 68.1 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 10.1 MB 61.2 MB/s 
     |████████████████████████████████| 144 kB 75.6 MB/s 
     |████████████████████████████████| 181 kB 69.6 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 

     |████████████████████████████████| 88 kB 6.6 MB/s 
  Created wheel for PyTDC: filename=PyTDC-0.3.6-py3-none-any.whl size=120882 sha256=112a80860100ffb443567b9266d422c7861cea6e92f72a146571494aab7d4162
  Stored in directory: /root/.cache/pip/wheels/c3/54/29/38349b4cf57cda21a1493f61721a6d72b232061f7665102d47
Successfully built PyTDC


In [ ]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn import metrics

In [ ]:
from tdc.single_pred import Yields
data = Yields(name = 'Buchwald-Hartwig')
split = data.get_split(method = 'random', seed = 42, frac = [0.7, 0.3,0.0])

Downloading...
100%|██████████| 16.4M/16.4M [00:01<00:00, 14.6MiB/s]
Loading...
Done!


In [ ]:
train = split['train']
valid  = split['valid']

In [ ]:
def data_cleaner(data):
    final_smiles = []
    y = list(data['Y'])
    for i in range(data.shape[0]):
        if data['Reaction'][i]['catalyst'] == '':
            smiles = data['Reaction'][i]['reactant']+'>>'+data['Reaction'][i]['product']
            final_smiles.append(smiles)
        else:
            smiles = data['Reaction'][i]['reactant']+'.'+data['Reaction'][i]['catalyst']+'>>'+data['Reaction'][i]['product']
            final_smiles.append(smiles)
    final_dict = {'text':final_smiles, 'labels':y}
    return final_dict

In [ ]:
train_df = pd.DataFrame(data_cleaner(train))
valid_df = pd.DataFrame(data_cleaner(valid))

In [ ]:
model = ClassificationModel('roberta', 'DeepChem/ChemBERTa-77M-MLM', args = {
'evaluate_during_training' : True,
'evaluate_during_training_verbose' : True,
'evaluate_during_training_steps' : 200,
'use_cached_eval_features ':True,
'use_early_stopping': True,
'regression':True,
'early_stopping_delta': 0.0001,
'early_stopping_metric':'eval_loss',
'early_stopping_metric_minimize' : True,
'early_stopping_patience' : 40,
'max_seq_length' : 512,
'num_train_epochs' : 40,
'train_batch_size' : 32,
'learning_rate' : 4e-5,
'reprocess_input_data': True, 
'overwrite_output_dir': True,
'auto_weights': True
}, num_labels=1,)

Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepChem/ChemBERTa-77M-MLM were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MLM and are newly initialized: ['classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420 [00:00<?, ?B/s]

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [10]:
out = model.train_model(train_df, eval_df=valid_df,args={'wandb_project': 'Buchwald-Hartwig'}, acc = metrics.r2_score)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/38759 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_roberta_512_1_2


Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

  0%|          | 0/16611 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_512_1_2
INFO:simpletransformers.classification.classification_model:{'acc': 0.968001122537702, 'eval_loss': 0.0023605480510968473}
INFO:simpletransformers.classification.classification_model: No improvement in eval_loss
INFO:simpletransformers.classification.classification_model: Current step: 13
INFO:simpletransformers.classification.classification_model: Early stopping patience: 40
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/16611 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_512_1_2
INFO:simpletransformers.classification.classification_model:{'acc': 0.9683416260335758, 'eval_loss': 0.002335664393655713}
INFO:simpletransformers.classification.classification_model: No improvement in eval_loss
INFO:simpletransformers.classification.classification_model: Current step: 14
INFO:simpletransformers.classification.classification_model: Early stopping patience: 40


KeyboardInterrupt: ignored

In [11]:
model = ClassificationModel(
    "roberta", "/content/outputs/best_model",args={'wandb_project': 'Buchwald-Hartwig'}
)

In [12]:
result, model_outputs, wrong_predictions = model.eval_model(valid_df,acc=metrics.r2_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/16611 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_roberta_512_1_2


Running Evaluation:   0%|          | 0/2077 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'acc': 0.9728798796870455, 'eval_loss': 0.0020007493180831735}


{'acc': 0.9728798796870455, 'eval_loss': 0.0020007493180831735}


In [13]:
!sudo apt-get install git-lfs
!git lfs install
!git config --global credential.helper store

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.


In [14]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [15]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/outputs/best_model")
model.push_to_hub("Parsa/Buchwald-Hartwig-Yield-prediction")

tokenizer = AutoTokenizer.from_pretrained("/content/outputs/best_model")
tokenizer.push_to_hub("Parsa/Buchwald-Hartwig-Yield-prediction")

Cloning https://huggingface.co/Parsa/Buchwald-Hartwig-Yield-prediction into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/13.1M [00:00<?, ?B/s]

To https://huggingface.co/Parsa/Buchwald-Hartwig-Yield-prediction
   25d5acc..1cf0584  main -> main

   25d5acc..1cf0584  main -> main

To https://huggingface.co/Parsa/Buchwald-Hartwig-Yield-prediction
   1cf0584..8de0fba  main -> main

   1cf0584..8de0fba  main -> main



'https://huggingface.co/Parsa/Buchwald-Hartwig-Yield-prediction/commit/8de0fba2ffb3e11faf5f2ab058043a1ebff3267f'